In [1]:
# 회귀
# 출력의 개수를 1개로
# 손실함수를 MSE나 기타 등등..
# 데이터셋 과 데이터 로드를 커스텀하게 정의해서 사용
# 나머지는 동일한 패턴으로 ... 학습 평가

In [2]:
# from sklearn.datasets import load_boston

In [3]:
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import torch

# 데이터 프레임
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

class BostonDataSet(Dataset):
    def __init__(self,X,y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).view(-1,1)
    def __len__(self):
        return len(self.X)
    def __getitem__(self,idx):
        return self.X[idx], self.y[idx]

In [4]:
X_dataset = BostonDataSet(data, target)  # 데이터를 X, y 를 한쌍으로 묶어
X_train_loader = DataLoader(X_dataset ,batch_size=32,shuffle=True)

In [5]:
# 회귀 모델 정의
import torch.nn as nn
class BostonRegression(nn.Module):
    def __init__(self, input_dim):
        super(BostonRegression,self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
        )
    def forward(self, x):
        return self.model(x)

In [6]:
from torch.optim import Adam
model = BostonRegression(data.shape[1])
criterion = nn.MSELoss()
optim = Adam(model.parameters(), lr = 1e-3)

In [7]:
from tqdm import tqdm
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
epochs = 100
# 학습루프
for epoch in range(epochs):
    tqdm_obj = tqdm(X_train_loader,desc=f'epoch : {epoch+1}/{epochs}')
    loss_lists = 0
    for data, label in tqdm_obj:
        optim.zero_grad()
        preds = model(data.to(device))
        loss = criterion(preds, label.to(device))
        loss_lists += loss.item()
        loss.backward()
        optim.step()    
        tqdm_obj.set_postfix({'loss' : f'{loss.item():.4f}'})
    avg_loss = loss_lists / len(X_train_loader)
print(f'epoch : {epoch+1} : avg loss : {avg_loss:.4f}')
        

torch.save(model.state_dict(), 'bostonRegression.pth') 

epoch : 100/100: 100%|██████████| 16/16 [00:00<00:00, 271.60it/s, loss=32.0731]

epoch : 100 : avg loss : 27.2787


In [8]:
from sklearn.metrics import r2_score
# 평가
model.load_state_dict(torch.load('bostonRegression.pth',map_location=device,weights_only=True))
# 예측
# 평가 루프
model.eval()  # 평가 모드로 전환 (dropout, batchnorm 등 비활성화)
total_mse = 0

criterion = nn.MSELoss()
r2scores = 0
with torch.no_grad():  # 그래디언트 계산 비활성화
    for data, label in tqdm(X_train_loader, desc="Evaluating"):
        data, label = data.to(device), label.to(device)
        preds = model(data)
        r2scores += r2_score(label.cpu().detach().numpy(), preds.cpu().detach().numpy())
        mse = criterion(preds, label)        
        total_mse += mse.item()
print(f"Test Loss: {total_mse / len(X_train_loader)} r2 score : {r2scores/len(X_train_loader)}")

Evaluating: 100%|██████████| 16/16 [00:00<00:00, 889.90it/s]

Test Loss: 21.516812682151794 r2 score : 0.7200615480542183
